In [1]:
import numpy as np
from numpy import dot, kron, outer, array, zeros, diag
from numpy.random import uniform
from functools import reduce
from itertools import product
from sympy import Symbol, simplify, trigsimp, Matrix, sin, cos, trace, im, exp, sqrt
from scipy.linalg import eigh, norm, det
from sympy.physics.quantum import TensorProduct, OuterProduct
from sympy.physics.vector import outer

In [2]:
s0 = array([1, 0])
s1 = array([0, 1])
X = np.array([[0.,1.],
              [1.,0.]]) # X Pauli matrix
Y = np.array([[0.,-1.j],
              [1.j, 0.]]) # Y Pauli matrix
Z = np.array([[1., 0.],
              [0.,-1.]]) # Z Pauli matrix
I = np.array([[1.,0.],
              [0.,1.]]) # 2x2 identity matrix
P0 = (I + Z)/2
P1 = (I - Z)/2
PP = (I + X)/2
PM = (I - X)/2
RP = np.outer(np.array([-1j, 1])/np.sqrt(2), np.array([1j, 1])/np.sqrt(2))
RM = np.outer(np.array([1j, 1])/np.sqrt(2), np.array([-1j, 1])/np.sqrt(2))

# Depolarizing

## Expectation

In [3]:
a = Symbol("a")
r1, r2, r3 = Symbol("r_1"), Symbol("r_2"), Symbol("r_3")
h0, h1, h2, h3 = Symbol("h_0"), Symbol("h_1"), Symbol("h_2"), Symbol("h_3")
x0, x1 = Symbol("x_0"), Symbol("x_1")

R = Matrix(1/2*(I + r1*X + r2*Y + r3*Z))
H = Matrix(h0*I + h1*X + h2*Y + h3*Z)

Ra = (1 - a)*R + a*I/2

In [4]:
expec = trace(Ra@H)
simplify(expec)

-1.0*a*h_1*r_1 - 1.0*a*h_2*r_2 - 1.0*a*h_3*r_3 + 1.0*h_0 + 1.0*h_1*r_1 + 1.0*h_2*r_2 + 1.0*h_3*r_3

In [5]:
simplify(expec.subs([(h0, 1), 
                     (h1, -1), 
                     (h2, h2), 
                     (h3, h3), 
                     (r1, 1), 
                     (r2, 0), 
                     (r3, 0)]))

1.0*a

In [6]:
simplify(expec.subs([(h0, 1), 
                     (h1, -1/(3*r1)), 
                     (h2, -1/(3*r2)), 
                     (h3, -1/(3*r3))]))

1.0*a + 1.11022302462516e-16

In [7]:
var = trace(Ra@H@H) - trace(Ra@H)**2
simplify(var)

-2.0*a*h_0*h_1*r_1 - 2.0*a*h_0*h_2*r_2 - 2.0*a*h_0*h_3*r_3 + 1.0*h_0**2 + 2.0*h_0*h_1*r_1 + 2.0*h_0*h_2*r_2 + 2.0*h_0*h_3*r_3 + 1.0*h_1**2 + 1.0*h_2**2 + 1.0*h_3**2 - 0.25*(-2*a*h_1*r_1 - 2*a*h_2*r_2 - 2*a*h_3*r_3 + 2*h_0 + 2*h_1*r_1 + 2*h_2*r_2 + 2*h_3*r_3)**2

In [8]:
simplify(var.subs([(h0, 1), 
                   (h1, -1), 
                   (h2, h2), 
                   (h3, h3), 
                   (r1, 1), 
                   (r2, 0), 
                   (r3, 0)]))

-1.0*a**2 + 2.0*a + 1.0*h_2**2 + 1.0*h_3**2

### In terms of the spectrum

In [9]:
R = Matrix(1/2*(I + r1*X + r2*Y + r3*Z))
H = Matrix(x0*P0 + x1*P1)

Ra = (1 - a)*R + a*I/2

res = trace(Ra@H)
res

1.0*x_0*(0.5*a + (1 - a)*(0.5*r_3 + 0.5)) + 1.0*x_1*(0.5*a + (0.5 - 0.5*r_3)*(1 - a))

In [10]:
simplify(res.subs([(x0, 1 - 1/r3), 
                   (x1, 1 + 1/r3)]))

1.0*a

# $z$-rotation

## degree 1

In [11]:
a = Symbol("a")
r1, r2, r3 = Symbol("r_1"), Symbol("r_2"), Symbol("r_3")
h0, h1, h2, h3 = Symbol("h_0"), Symbol("h_1"), Symbol("h_2"), Symbol("h_3")

In [12]:
R = Matrix(1/2*(I + X))
# R = Matrix(1/2*(I + r1*X + r2*Y + r3*Z))
H = Matrix(h0*I + h1*X + h2*Y + h3*Z)
Uz = Matrix(cos(a/2)*I - 1j*sin(a/2)*Z)
Uz_dg = Matrix(cos(a/2)*I + 1j*sin(a/2)*Z)

In [13]:
Ra = simplify(Uz@R@Uz_dg)

res = trace(Ra@H)
res

1.0*h_0 + 0.5*(1.0*h_1 - 1.0*I*h_2)*exp(I*a) + 0.5*(1.0*h_1 + 1.0*I*h_2)*exp(-I*a)

In [14]:
trace(Ra@X)

0.5*exp(I*a) + 0.5*exp(-I*a)

## Fourier coefficients

In [15]:
a = Symbol("a")
r1, r2, r3 = Symbol("r_1"), Symbol("r_2"), Symbol("r_3")

P01 = np.outer(s0, s1)
P10 = np.outer(s1, s0)

Uz = Matrix(cos(a/2)*I - 1j*sin(a/2)*Z)
Uz_dg = Matrix(cos(a/2)*I + 1j*sin(a/2)*Z)

# R = Matrix(1/2*(I + r1*X + r2*Y + r3*Z))
R = Matrix(1/2*(I + X))

In [16]:
Matrix(simplify(Uz@R@Uz_dg))

Matrix([
[         0.5, 0.5*exp(-I*a)],
[0.5*exp(I*a),           0.5]])

In [17]:
c = 4

Rnc = simplify(Uz@R@Uz_dg)
for j in range(c - 1):
    Rnc = simplify(TensorProduct(Matrix(simplify(Uz@R@Uz_dg)), Rnc))

for j in range(1, c + 1):
    mul = 2**j*(-1)**j/j
    op = 1j*(reduce(kron, [P01]*j + [I]*(c - j)) - reduce(kron, [P10]*j + [I]*(c - j))) *mul
    res = simplify(trace(Rnc@op))
    print(res)

2.0*sin(a)
-1.0*sin(2*a)
0.666666666666667*sin(3*a)
-0.5*sin(4*a)


# Amplitude-damping

In [18]:
g = Symbol("\gamma")
r1, r2, r3 = Symbol("r_1"), Symbol("r_2"), Symbol("r_3")
h0, h1, h2, h3 = Symbol("h_0"), Symbol("h_1"), Symbol("h_2"), Symbol("h_3")
x0, x1 = Symbol("x_0"), Symbol("x_1")

In [19]:
R = Matrix(1/2*(I + r1*X + r2*Y + r3*Z))
H = Matrix(h0*I + h1*X + h2*Y + h3*Z)

K1 =  Matrix([[1,           0],
              [0, sqrt(1 - g)]])
K2 =  Matrix([[0, sqrt(g)],
              [0,       0]])

Rn = K1@R@K1.T + K2@R@K2.T

In [20]:
simplify(Rn.subs([ (r1, 1), 
                   (r2, 0), 
                   (r3, 0)]))

Matrix([
[    0.5*\gamma + 0.5, 0.5*sqrt(1 - \gamma)],
[0.5*sqrt(1 - \gamma),     0.5 - 0.5*\gamma]])

In [21]:
expec = trace(Rn@H)
simplify(expec)

-1.0*\gamma*h_3*r_3 + 1.0*\gamma*h_3 + 1.0*h_0 + 1.0*h_1*r_1*sqrt(1 - \gamma) + 1.0*h_2*r_2*sqrt(1 - \gamma) + 1.0*h_3*r_3

In [22]:
simplify(expec.subs([(h0, r3/(r3 - 1)), 
                     (h1, 0), 
                     (h2, 0), 
                     (h3, -1/(r3 - 1))]))

1.0*\gamma

In [23]:
simplify(expec.subs([(h0, r3/(r3 - 1)), 
                     (h1, 1/r1), 
                     (h2, -1/r2), 
                     (h3, -1/(r3 - 1))]))

1.0*\gamma

In [24]:
simplify(expec.subs([(h0, 0), 
                     (h1, 0.5/r1), 
                     (h2, 0.5/r2), 
                     (h3, 0)]))

1.0*sqrt(1 - \gamma)

In [25]:
disp = trace(Rn@H@H) - trace(Rn@H)**2
simplify(disp.subs([(h0, 0), 
                   (h1, 0), 
                   (h2, h2), 
                   (h3, 1),
                   (r1, 1),
                   (r2, 0),
                   (r3, 0)]))

-1.0*\gamma**2 + 1.0*h_2**2 + 1.0

In [26]:
disp = trace(Rn@H@H) - trace(Rn@H)**2
simplify(disp.subs([(h0, 0), 
                   (h1, 0), 
                   (h2, 0), 
                   (h3, 1),
                   (r1, 1),
                   (r2, 0),
                   (r3, 0)]))

1.0 - 1.0*\gamma**2

### Engineered Hamiltonian

#### Z eigenbasis

In [27]:
R = Matrix(1/2*(I + X))
H = Matrix(h0*I + h1*X + h2*Y + h3*Z)

K1 =  Matrix([[1,           0],
              [0, sqrt(1 - g)]])
K2 =  Matrix([[0, sqrt(g)],
              [0,       0]])

Rn = K1@R@K1.T + K2@R@K2.T
res = trace(Rn@H)
simplify(res)

1.0*\gamma*h_3 + 1.0*h_0 + 1.0*h_1*sqrt(1 - \gamma)

In [28]:
H = Matrix(x0*P0 + x1*P1) # Z eigenbasis
res = trace(Rn@H)
simplify(res)

0.5*x_0*(\gamma + 1) - 0.5*x_1*(\gamma - 1)

In [29]:
simplify(res.subs([(x0, 1),
                   (x1, -1)
                  ]))

1.0*\gamma

# Generalized amplitude-damping

## degree 1

In [30]:
g = Symbol("\gamma")
N = Symbol("N")
r1, r2, r3 = Symbol("r_1"), Symbol("r_2"), Symbol("r_3")
h0, h1, h2, h3 = Symbol("h_0"), Symbol("h_1"), Symbol("h_2"), Symbol("h_3")
x0, x1 = Symbol("x_0"), Symbol("x_1")

In [31]:
R = Matrix(1/2*(I + r1*X + r2*Y + r3*Z))
H = Matrix(h0*I + h1*X + h2*Y + h3*Z)

K1 =  Matrix([[1,           0],
              [0, sqrt(1 - g)]])*sqrt(1 - N)
K2 =  Matrix([[0, sqrt(g*(1 - N))],
              [0,               0]])
K3 =  Matrix([[sqrt(1 - g), 0],
              [          0, 1]])*sqrt(N)
K4 =  Matrix([[0,         0],
              [sqrt(g*N), 0]])

Ra = K1@R@K1.T + K2@R@K2.T  + K3@R@K3.T  + K4@R@K4.T 
res = trace(Ra@H)
simplify(res)

-2.0*N*\gamma*h_3 - 1.0*\gamma*h_3*r_3 + 1.0*\gamma*h_3 + 1.0*h_0 + 1.0*h_1*r_1*sqrt(1 - \gamma) + 1.0*h_2*r_2*sqrt(1 - \gamma) + 1.0*h_3*r_3

### Fixed $N$

#### Non-linear in $\gamma$

In [32]:
simplify(res.subs([(h0, 0), 
                   (h1, 1/(2*r1)), 
                   (h2, 1/(2*r2)), 
                   (h3, 0)]))

1.0*sqrt(1 - \gamma)

#### Linear in $\gamma$

In [33]:
simplify(res.subs([(h0, r3/(2*N + r3 - 1)), 
                   (h1, 0), 
                   (h2, 0), 
                   (h3, -1/(2*N + r3 - 1))]))

1.0*\gamma

In [34]:
simplify(r3/(2*N + r3 - 1)*I + -1/(2*N + r3 - 1)*Z)

[[1.0*(r_3 - 1)/(2*N + r_3 - 1), 0], [0, 1.0*(r_3 + 1)/(2*N + r_3 - 1)]]

#### If can control the input state

In [35]:
simplify(res.subs([(h0, 0), (h1, 1/sqrt(2)), (h2, 1/sqrt(2)), (h3, 0),
                            (r1, 1/sqrt(2)), (r2, 1/sqrt(2)), (r3, 0)]))

1.0*sqrt(1 - \gamma)

#### $N \neq \frac{1}{2}$

In [36]:
simplify(res.subs([(h0, 0), 
                   (h1, 0),
                   (h2, h2),
                   (h3, 1/(1 - 2*N)),
                   (r1, r1),
                   (r2, 0),
                   (r3, 0)]))

1.0*\gamma

#### $N = \frac{1}{2}$

In [37]:
simplify(res.subs([
                   (N, 1/2),
                   (h0, 1), 
                   (h1, 0),
                   (h2, 0),
                   (h3, -1),
                   (r1, 0),
                   (r2, 0),
                   (r3, 1)
                  ]))

1.0*\gamma

#### Fixed $\gamma$

In [38]:
simplify(res)

-2.0*N*\gamma*h_3 - 1.0*\gamma*h_3*r_3 + 1.0*\gamma*h_3 + 1.0*h_0 + 1.0*h_1*r_1*sqrt(1 - \gamma) + 1.0*h_2*r_2*sqrt(1 - \gamma) + 1.0*h_3*r_3

In [39]:
simplify(res.subs([(h0, 0), 
                   (h1, 0), 
                   (h2, 0), 
                   (h3, h3)]))

h_3*(-2.0*N*\gamma - 1.0*\gamma*r_3 + 1.0*\gamma + 1.0*r_3)

### Engineered Hamiltonian

#### Z eigenbasis

In [40]:
H = Matrix(x0*P0 + x1*P1) # Z eigenbasis
res = trace(Ra@H)
simplify(res)

-0.5*x_0*(N*(\gamma - 1)*(r_3 + 1) - \gamma*(N - 1)*(r_3 - 1) + (N - 1)*(r_3 + 1)) - 0.5*x_1*(-N*\gamma*(r_3 + 1) + N*(r_3 - 1) + (N - 1)*(\gamma - 1)*(r_3 - 1))

##### $r_3 \neq 0$, $N \neq \frac{1}{2}$

In [41]:
simplify(res.subs([(x0, (r3 - 1)/(2*N + r3 - 1)),
                   (x1, (r3 + 1)/(2*N + r3 - 1))
                  ]))

1.0*\gamma

#### X eigenbasis

In [42]:
H = Matrix(x0*PP + x1*PM) # X eigenbasis
res = trace(Ra@H)
simplify(res)

0.5*r_1*x_0*sqrt(1 - \gamma) - 0.5*r_1*x_1*sqrt(1 - \gamma) + 0.5*x_0 + 0.5*x_1

##### $r_1 \neq 0$

In [43]:
simplify(res.subs([(x0, 1/r1), (x1, -1/r1)]))

1.0*sqrt(1 - \gamma)

#### Y eigenbasis

In [44]:
H = Matrix(x0*RP + x1*RM) # X eigenbasis
res = trace(Ra@H)
simplify(res)

0.5*r_2*x_0*sqrt(1 - \gamma) - 0.5*r_2*x_1*sqrt(1 - \gamma) + 0.5*x_0 + 0.5*x_1

##### $r_2 \neq 0$

In [45]:
simplify(res.subs([(x0, 1/r2), (x1, -1/r2)]))

1.0*sqrt(1 - \gamma)

## degree 2

In [46]:
g = Symbol("\gamma")
N = Symbol("N")
r1, r2, r3 = Symbol("r_1"), Symbol("r_2"), Symbol("r_3")
x0, x1 = Symbol("x_0"), Symbol("x_1")
hs = [Symbol("h_" + str(i) + str(j) + "") for i in range(4) for j in range(4)]

paulies = [reduce(kron, el) for el in product([I, X, Y, Z], repeat=2)]
H = zeros([4, 4], dtype=complex)
for h, P in zip(hs, paulies):
    H = H + h*P
H = Matrix(H)

K1 =  Matrix([[1,           0],
              [0, sqrt(1 - g)]])*sqrt(1 - N)
K2 =  Matrix([[0, sqrt(g*(1 - N))],
              [0,               0]])
K3 =  Matrix([[sqrt(1 - g), 0],
              [          0, 1]])*sqrt(N)
K4 =  Matrix([[0,         0],
              [sqrt(g*N), 0]])

R = Matrix(1/2*(I + r1*X + r2*Y + r3*Z))
Rn = K1@R@K1.T + K2@R@K2.T  + K3@R@K3.T  + K4@R@K4.T
Rn = TensorProduct(Rn, Rn)

res = simplify(trace(Rn@H))
res

4.0*N**2*\gamma**2*h_33 + 4.0*N*\gamma**2*h_33*r_3 - 4.0*N*\gamma**2*h_33 - 2.0*N*\gamma*h_03 - 2.0*N*\gamma*h_13*r_1*sqrt(1 - \gamma) - 2.0*N*\gamma*h_23*r_2*sqrt(1 - \gamma) - 2.0*N*\gamma*h_30 - 2.0*N*\gamma*h_31*r_1*sqrt(1 - \gamma) - 2.0*N*\gamma*h_32*r_2*sqrt(1 - \gamma) - 4.0*N*\gamma*h_33*r_3 + 1.0*\gamma**2*h_33*r_3**2 - 2.0*\gamma**2*h_33*r_3 + 1.0*\gamma**2*h_33 - 1.0*\gamma*h_03*r_3 + 1.0*\gamma*h_03 - 1.0*\gamma*h_11*r_1**2 - 1.0*\gamma*h_12*r_1*r_2 - 1.0*\gamma*h_13*r_1*r_3*sqrt(1 - \gamma) + 1.0*\gamma*h_13*r_1*sqrt(1 - \gamma) - 1.0*\gamma*h_21*r_1*r_2 - 1.0*\gamma*h_22*r_2**2 - 1.0*\gamma*h_23*r_2*r_3*sqrt(1 - \gamma) + 1.0*\gamma*h_23*r_2*sqrt(1 - \gamma) - 1.0*\gamma*h_30*r_3 + 1.0*\gamma*h_30 - 1.0*\gamma*h_31*r_1*r_3*sqrt(1 - \gamma) + 1.0*\gamma*h_31*r_1*sqrt(1 - \gamma) - 1.0*\gamma*h_32*r_2*r_3*sqrt(1 - \gamma) + 1.0*\gamma*h_32*r_2*sqrt(1 - \gamma) - 2.0*\gamma*h_33*r_3**2 + 2.0*\gamma*h_33*r_3 + 1.0*h_00 + 1.0*h_01*r_1*sqrt(1 - \gamma) + 1.0*h_02*r_2*sqrt(1 - 

In [47]:
simplify(res.subs([
                   (r3, 1),
                   (r1, 0),
                   (r2, 0),
                   (N, 0),
                  ]))

1.0*h_00 + 1.0*h_03 + 1.0*h_30 + 1.0*h_33

#### $r_1 \neq 0$, $r_2 \neq 0$

In [48]:
muls = uniform(-1, 1, 4); muls = muls/sum(muls)
# muls = [Symbol("a"), Symbol("b"), Symbol("c"), Symbol("d")]
simplify(res.subs([
                   (hs[0], 1), # h00
                   (hs[1], 0), # h01*r1*sqrt(1 - g)
                   (hs[2], 0), # h02*r2*sqrt(1 - g)
                   (hs[3], 0), # h03*(-2*N*g - g*r3 + g + r3)
                   (hs[4], 0), # h10*r1*sqrt(1 - g)
                   (hs[5], -muls[0]/(r1**2)), # h11*r1^2*(1 - g)
                   (hs[6], -muls[1]/(r1*r2)), # h12*r1*r2*(1 - g)
                   (hs[7], 0), # h13*r1*sqrt(1 - g)*(-2*N*g - g*r3 + g + r3)
                   (hs[8], 0), # h20*r2*sqrt(1 - g)
                   (hs[9], -muls[2]/(r1*r2)), # h21*r1*r2*(1 - g) 
                   (hs[10], -muls[3]/(r2**2)), # h22*r2^2*(1 - g)
                   (hs[11], 0), # h23*r2*sqrt(1 - g)*(-2*N*g - g*r3 + g + r3)
                   (hs[12], 0), # h30*(-2*N*g - g*r3 + g + r3)
                   (hs[13], 0), # h31*r1*sqrt(1 - g)*(-2*N*g - g*r3 + g + r3)
                   (hs[14], 0), # h32*r2*sqrt(1 - g)*(-2*N*g - g*r3 + g + r3)
                   (hs[15], 0), # h33*(N^2*g^2 + N*g^2*r3 - N*g^2 - N*g*r3 + g^2*r3^2 - g^2*r3 + g^2 - g*r3^2 + g*r3 + r3^2)
                  ]))

1.0*\gamma - 5.55111512312578e-17

#### $r_3 \neq 0$

In [49]:
muls = uniform(-1, 1, 2); muls = muls/sum(muls)
# muls = [Symbol("a"), Symbol("b")]
simplify(res.subs([
                   (hs[0], r3/(2*N + r3 - 1)), # h00
                   (hs[1], 0), # h01*r1*sqrt(1 - g)
                   (hs[2], 0), # h02*r2*sqrt(1 - g)
                   (hs[3], -muls[0]/(2*N + r3 - 1)),  # h03*(-2*N*g - g*r3 + g + r3)
                   (hs[4], 0), # h10*r1*sqrt(1 - g)
                   (hs[5], 0), # h11*r1^2*(1 - g)
                   (hs[6], 0), # h12*r1*r2*(1 - g)
                   (hs[7], 0), # h13*r1*sqrt(1 - g)*(-2*N*g - g*r3 + g + r3)
                   (hs[8], 0), # h20*r2*sqrt(1 - g)
                   (hs[9], 0), # h21*r1*r2*(1 - g) 
                   (hs[10], 0), # h22*r2^2*(1 - g)
                   (hs[11], 0), # h23*r2*sqrt(1 - g)*(-2*N*g - g*r3 + g + r3)
                   (hs[12], -muls[1]/(2*N + r3 - 1)), # h30*(-2*N*g - g*r3 + g + r3)
                   (hs[13], 0), # h31*r1*sqrt(1 - g)*(-2*N*g - g*r3 + g + r3)
                   (hs[14], 0), # h32*r2*sqrt(1 - g)*(-2*N*g - g*r3 + g + r3)
                   (hs[15], 0), # h33*(N^2*g^2 + N*g^2*r3 - N*g^2 - N*g*r3 + g^2*r3^2 - g^2*r3 + g^2 - g*r3^2 + g*r3 + r3^2)
                  ]))

1.0*\gamma

### Engineered Hamiltonian

#### $r_3 \neq 0$ and $N \neq \frac{1}{2}$

In [50]:
H = Matrix(x0*(kron(P0, P0) + kron(P0, P1)) + x1*(kron(P1, P0) + kron(P1, P1))) # Z eigenbasis
res = trace(Rn@H)
simplify(res)

-1.0*N*\gamma*x_0 + 1.0*N*\gamma*x_1 - 0.5*\gamma*r_3*x_0 + 0.5*\gamma*r_3*x_1 + 0.5*\gamma*x_0 - 0.5*\gamma*x_1 + 0.5*r_3*x_0 - 0.5*r_3*x_1 + 0.5*x_0 + 0.5*x_1

In [51]:
simplify(res.subs([(x0, (-1 + r3)/(2*N + r3 - 1)),
                   (x1, (1 + r3)/(2*N + r3 - 1))
                  ]))

1.0*\gamma

#### $r_1 \neq 0$

In [52]:
H = Matrix(x0*(kron(PP, PP) + kron(PM, PM)) + x1*(kron(PP, PM) + kron(PM, PP))) # X eigenbasis
res = trace(Rn@H)
simplify(res)

-0.5*\gamma*r_1**2*x_0 + 0.5*\gamma*r_1**2*x_1 + 0.5*r_1**2*x_0 - 0.5*r_1**2*x_1 + 0.5*x_0 + 0.5*x_1

In [53]:
simplify(res.subs([(x0, 1 - 1/r1**2), (x1, 1 + 1/r1**2)]))

1.0*\gamma

#### $r_2 \neq 0$

In [54]:
H = Matrix(x0*(kron(RP, RP) + kron(RM, RM)) + x1*(kron(RP, RM) + kron(RM, RP))) # Y eigenbasis
res = trace(Rn@H)
simplify(res)

-0.5*\gamma*r_2**2*x_0 + 0.5*\gamma*r_2**2*x_1 + 0.5*r_2**2*x_0 - 0.5*r_2**2*x_1 + 0.5*x_0 + 0.5*x_1

In [55]:
simplify(res.subs([(x0, 1 - 1/r2**2), (x1, 1 + 1/r2**2)]))

1.0*\gamma